In [1]:
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import glob,os,sys
from tqdm import tqdm
import proplot as plot
import json,pickle
import dask.array as da
import gc
from sklearn.decomposition import PCA
sys.path.insert(1, '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/')
from tools import derive_var,read_and_proc,preproc_noensemble
from tools.mlr import mlr
from tools.preprocess import do_eof,preproc_maria
%matplotlib inline
plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

import time
import numpy as np
from dask import delayed
from dask.distributed import Client
client = Client(processes=True, threads_per_worker=1,n_workers=2)

/tmp/ipykernel_1689686/1245973198.py:20: ProplotWarning: rc setting 'linewidth' was renamed to 'meta.width' in version 0.8.
  plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',


In [2]:
path = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/'
suffix = '_smooth_preproc_dict1b_g'

haiyan_u = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['u'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
haiyan_v = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['v'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
haiyan_w = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['w'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
haiyan_qv = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['qv'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
haiyan_theta = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['theta'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
haiyan_heatsum = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['heatsum'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
haiyan_hdia = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['hdia'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
haiyan_rad = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['rad'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
haiyan_ir = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['ir'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]

100%|██████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:29<00:00,  5.86s/it]


In [5]:
def long_MariaExps(array=None):
    haiyan_temparray = [array[0][12:,:],array[1][36:,:],array[2][60:,:],array[3][96:,:],array[4][36:,:]]
    del array
    gc.collect()
    return np.concatenate(haiyan_temparray,axis=0)

haiyan_ua,haiyan_va,haiyan_wa = long_MariaExps(haiyan_u),long_MariaExps(haiyan_v),long_MariaExps(haiyan_w)
haiyan_qva,haiyan_thetaa,haiyan_heatsuma = long_MariaExps(haiyan_qv),long_MariaExps(haiyan_theta),long_MariaExps(haiyan_heatsum)
haiyan_hdiaa,haiyan_rada,haiyan_ira = long_MariaExps(haiyan_hdia),long_MariaExps(haiyan_rad),long_MariaExps(haiyan_ir)

In [6]:
def long_MariaExps(array=None):
    haiyan_temparray = [array[0][12:,:],array[1][36:,:],array[2][60:,:],array[3][96:,:],array[4][36:,:]]
    del array
    gc.collect()
    return np.concatenate(haiyan_temparray,axis=0)

def build_a_xarray_dataset(ds=None,varname=None,varfile=None,dims=None,coords=None):
    def build_xa(arraydata=None,arrayname=None):
        da = xr.DataArray(data=arraydata,dims=dims,coords=coords,name=arrayname)
        return da
    # Complete the code
    for varobj,varnamen in zip(varfile,varname):
        da = build_xa(varobj,varnamen)
        ds = xr.merge( [ds , da ] )
    return ds

dims = ['sample','flatarray']
coords = dict(sample=np.linspace(0,haiyan_ua.shape[0]-1,haiyan_ua.shape[0]),flatarray=np.linspace(0,haiyan_ua.shape[1]-1,haiyan_ua.shape[1]))

ds = xr.Dataset(coords=coords)
maria_data=build_a_xarray_dataset(ds=ds,varname=['u','v','w','theta','heatsum','hdia','rad','qv','ir'],\
                                   varfile=[haiyan_ua,haiyan_va,haiyan_wa,haiyan_thetaa,haiyan_heatsuma,haiyan_hdiaa,haiyan_rada,haiyan_qva,haiyan_ira],dims=dims,coords=coords)
del haiyan_ua,haiyan_va,haiyan_wa,haiyan_thetaa,haiyan_heatsuma,haiyan_hdiaa,haiyan_rada,haiyan_ira
gc.collect()

0

In [7]:
from sklearn.decomposition import IncrementalPCA
def fit_cheap_pca(n_batches=None,n_comps=None,var=None):
    #n_batches = 20
    inc_pca = IncrementalPCA(n_components=n_comps)
    for X_batch in tqdm(np.array_split(var.data,n_batches)):
        inc_pca.partial_fit(X_batch)
    return inc_pca

In [10]:
pca_store = {}
for varnamex in (['u','v','w','theta','heatsum','hdia','rad','qv','ir']):#['u','v','w','theta','heatsum','hdia','rad','qv']):
    pca_store[varnamex] = fit_cheap_pca(n_batches=3,n_comps=150,var=maria_data[varnamex])

100%|██████████████████████████████████████████████████████████████████████████████████████| 3/3 [03:54<00:00, 78.08s/it]


In [13]:
folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/maria/processed/'
read_and_proc.save_to_pickle(folderpath+'PCA/PCAdict2',pca_store,'PICKLE')

#### Intermediate

In [9]:
def long_MariaExps(array=None):
    haiyan_temparray = [array[0][12:,:],array[1][36:,:],array[2][60:,:],array[3][96:,:],array[4][36:,:]]
    del array
    gc.collect()
    return np.concatenate(haiyan_temparray,axis=0)

def to_azim(array=None,shape=[39,360,200]):
    def _to_azim(array=None):
        arrayn = array.reshape(array.shape[0],shape[0],shape[1],shape[2])
        return np.nanmean(arrayn,axis=2).reshape(array.shape[0],shape[0]*shape[2])
    return [_to_azim(obj) for obj in array]

def build_a_xarray_dataset(ds=None,varname=None,varfile=None,dims=None,coords=None):
    def build_xa(arraydata=None,arrayname=None):
        da = xr.DataArray(data=arraydata,dims=dims,coords=coords,name=arrayname)
        return da
    # Complete the code
    for varobj,varnamen in zip(varfile,varname):
        da = build_xa(varobj,varnamen)
        ds = xr.merge( [ds , da ] )
    return ds

from sklearn.decomposition import IncrementalPCA
def fit_cheap_pca(n_batches=None,n_comps=None,var=None):
    #n_batches = 20
    inc_pca = IncrementalPCA(n_components=n_comps)
    for X_batch in tqdm(np.array_split(var.data,n_batches)):
        inc_pca.partial_fit(X_batch)
    return inc_pca

In [12]:
haiyan_uazim,haiyan_vazim,haiyan_wazim = long_MariaExps(to_azim(haiyan_u)),long_MariaExps(to_azim(haiyan_v)),long_MariaExps(to_azim(haiyan_w))
haiyan_qvazim,haiyan_thetaazim,haiyan_heatsumazim = long_MariaExps(to_azim(haiyan_qv)),long_MariaExps(to_azim(haiyan_theta)),long_MariaExps(to_azim(haiyan_heatsum))
haiyan_hdiaazim,haiyan_radazim,haiyan_irazim = long_MariaExps(to_azim(haiyan_hdia)),long_MariaExps(to_azim(haiyan_rad)),long_MariaExps(to_azim(haiyan_ir))

In [13]:
dims = ['sample','flatarray']
coords = dict(sample=np.linspace(0,haiyan_uazim.shape[0]-1,haiyan_uazim.shape[0]),flatarray=np.linspace(0,haiyan_uazim.shape[1]-1,haiyan_uazim.shape[1]))

ds = xr.Dataset(coords=coords)
maria_data=build_a_xarray_dataset(ds=ds,varname=['u','v','w','theta','heatsum','hdia','rad','qv','ir'],\
                                   varfile=[haiyan_uazim,haiyan_vazim,haiyan_wazim,haiyan_thetaazim,haiyan_heatsumazim,haiyan_hdiaazim,haiyan_radazim,haiyan_qvazim,haiyan_irazim],dims=dims,coords=coords)
del haiyan_uazim,haiyan_vazim,haiyan_wazim,haiyan_thetaazim,haiyan_heatsumazim,haiyan_hdiaazim,haiyan_radazim,haiyan_irazim
gc.collect()

0

In [14]:
from sklearn.decomposition import IncrementalPCA
def fit_cheap_pca(n_batches=None,n_comps=None,var=None):
    #n_batches = 20
    inc_pca = IncrementalPCA(n_components=n_comps)
    for X_batch in tqdm(np.array_split(var.data,n_batches)):
        inc_pca.partial_fit(X_batch)
    return inc_pca

In [15]:
pca_store = {}
for varnamex in (['u','v','w','theta','heatsum','hdia','rad','qv','ir']):#['u','v','w','theta','heatsum','hdia','rad','qv']):
    pca_store[varnamex] = fit_cheap_pca(n_batches=3,n_comps=150,var=maria_data[varnamex])

100%|██████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  9.20it/s]


In [16]:
folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/maria/processed/intermediate/'
read_and_proc.save_to_pickle(folderpath+'PCA/PCAdict',pca_store,'PICKLE')